In [4]:
import numpy as np 
import pandas as pd
import requests
from groq import Groq
from datetime import datetime
import calendar
import matplotlib.pyplot as plt
import math
from openai import OpenAI
import time

In [5]:
contracts16 = pd.read_csv("/home/xikron/Projects/misc/data/blacktesting/c16.csv")
contracts16['Start Date']= pd.to_datetime(contracts16['Start Date'])
contracts16=contracts16.drop_duplicates(subset='Award ID').sort_values(by='Start Date')

In [6]:
contracts17 = pd.read_csv("/home/xikron/Projects/misc/data/blacktesting/c17.csv")
contracts17['Start Date']= pd.to_datetime(contracts17['Start Date'])
contracts17=contracts17.drop_duplicates(subset='Award ID').sort_values(by='Start Date')

In [7]:
contracts18 = pd.read_csv("/home/xikron/Projects/misc/data/blacktesting/c18.csv")
contracts18['Start Date']= pd.to_datetime(contracts18['Start Date'])
contracts18=contracts18.drop_duplicates(subset='Award ID').sort_values(by='Start Date')

In [8]:
def setLinkIntd (ticker, start, end, freq):
    start_str = start.strftime('%Y-%m-%d') if isinstance(start, pd.Timestamp) else start 
    end_str = end.strftime('%Y-%m-%d') if isinstance(end, pd.Timestamp) else end

    link = "https://financialmodelingprep.com/api/v3/historical-chart/" + freq + "/" + ticker + "?from=" + start_str + "&to=" + end_str + "&apikey=26srycwxWrFIhEuaZwic6mBdx7f4VjGT"
    return link


def setLinkEod (ticker, start, end):
    start_str = start.strftime('%Y-%m-%d') if isinstance(start, pd.Timestamp) else start
    end_str = end.strftime('%Y-%m-%d') if isinstance(end, pd.Timestamp) else end

    return "https://financialmodelingprep.com/api/v3/historical-price-full/" + ticker + "?from=" + start_str + "&to=" + end_str + "&apikey=26srycwxWrFIhEuaZwic6mBdx7f4VjGT"

In [9]:
response = requests.get(url=setLinkIntd("LMT", "2010-03-31", "2010-03-31", "5min"))
response1 = requests.get(url=setLinkEod("RTX", "2001-01-01", "2001-01-02"))

In [10]:
response1.json()['historical']
prices = pd.Series([entry['close'] for entry in response1.json()['historical']])        
print(prices)

0    22.19
dtype: float64


In [11]:
import uuid

class distinctId:
    @staticmethod
    def genId():
        return str(uuid.uuid4())

In [12]:
from dataclasses import dataclass

@dataclass
class transaction:
    contractID: int
    num: int


In [13]:
class Portfolio: 
    def __init__(self, thresh, startingConst, start=1000000000):
        self.id = distinctId.genId()  
        self.capital = start
        self.trades = {}
        self.sells = []
        self.buys = []
        self.mRThresh = thresh
        self.buySig = startingConst

    def logTrade(self, ticker, iD, shares): 
        if ticker not in self.trades:
            self.trades[ticker] = [transaction(iD, shares)]
        else: 
            self.trades[ticker].append(transaction(iD, shares)) 

    def buy(self, pps, ticker, shares, iD, buyDate):
        if self.capital >= pps * shares:  
            self.capital -= pps * shares
            self.logTrade(ticker, iD, shares)
            self.buys.append(f"B{shares}{ticker}{pps}D{buyDate}")
            print(f"B{shares}{ticker}{pps}D{buyDate}")
        else:
            print("Insufficient capital to buy shares!")

    def sell(self, pps, iD, ticker, sellDate):
        if ticker in self.trades:
            for t in self.trades[ticker]:
                if t.contractID == iD: 
                    shares = t.num
                    self.capital += shares * pps
                    self.sells.append(f"S{shares}{ticker}{pps}D{sellDate}")
                    print(f"S{shares}{ticker}{pps}D{sellDate}")
                    self.trades[ticker].remove(t) 
                    break  

In [14]:
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

def getBuyDay(contractDate, prev):
    uBday = CustomBusinessDay(calendar=USFederalHolidayCalendar())
    contractDate = pd.Timestamp(contractDate)
    buyDay = contractDate - prev * uBday
    return buyDay.strftime('%Y-%m-%d')


In [15]:
def meanReversion(prices, threshold, window=20):
    if len(prices) < window:
        print(f"Warning: Not enough data points for rolling window (got {len(prices)}, expected {window})")
        return False  

    sma = prices.rolling(window=window).mean()
    std_dev = prices.rolling(window=window).std()

    z_score = (prices - sma) / std_dev

    if len(z_score) > 0:
        if z_score.iloc[-1] > threshold:
            return True  
        elif z_score.iloc[-1] < -threshold:
            return True
    else:
        print("Error: 'z_score' is empty or invalid.")
    
    return False  

In [16]:
def getSellDay(contractDate, ticker, portfolio): 
    us_business_day = CustomBusinessDay(calendar=USFederalHolidayCalendar())

    contractDate = pd.Timestamp(contractDate)
    adjDate = contractDate if us_business_day.is_on_offset(contractDate) else contractDate + us_business_day
    adjDate += 1 * us_business_day  

    startDay = adjDate - pd.Timedelta(days=30)
    endDay = adjDate 

    response = requests.get(setLinkIntd(ticker, startDay.strftime('%Y-%m-%d'), endDay.strftime(('%Y-%m-%d')), "5min"))

    if response.status_code == 200:
        data = response.json()
        if len(data)==0: 
            try:
                response = requests.get(setLinkEod(ticker, startDay.strftime('%Y-%m-%d'), endDay.strftime(('%Y-%m-%d'))))
                data = response.json()['historical']
            except Exception as e: 
                print("Unexpected exception: "+ str(e))
                return None
        if len(data)!= 0:
            prices = pd.Series([entry['close'] for entry in data])        

            if meanReversion(prices, portfolio.mRThresh):
                return adjDate.strftime('%Y-%m-%d')  
            else:
                next_day = adjDate + us_business_day
                return getSellDay(next_day.strftime('%Y-%m-%d'), ticker, portfolio)  
        else: 
            return

In [17]:
import time 
def buy(ticker, pP, buyDate, id, portfolio): 
    response = requests.get(url=setLinkIntd(ticker, buyDate, buyDate, "5min"))
    for i in range (2): 
        if response.status_code == 200: 
            if len(response.json()) == 0: 
                response = requests.get(url=setLinkEod(ticker, buyDate, buyDate))
                try: 
                    pps = response.json()['historical'][0]['close']  
                except Exception as e: 
                    print("Response length " + str(len(response.json())) + " with unexpected exception: " + str(e))  
                    return False
            if len(response.json())>15:
                pps = response.json()[-15]["open"]

            if (pps!=None): 
                shares = math.floor(pP/pps)
                portfolio.buy(pps, ticker, shares, id, buyDate)
                
                return True 
            else: 
                return False
        elif response.status_code == 429: 
            time.sleep(15)
            continue
        else:
            return False
    return False 

In [18]:
def sell(ticker, sellDate, id, portfolio):
    response = requests.get(url=setLinkIntd(ticker, sellDate, sellDate, "5min"))
    if response.status_code == 200 and len(response.json()) >15: 
        pps = response.json()[-15]["open"]
        portfolio.sell(pps, id, ticker, sellDate)


In [19]:
def runPortfolio(contracts, mRSig, prev): 
    global allValueation, allBuys, allSells 
    portfolio = Portfolio(mRSig, prev)
    for i in range (len(contracts)):
        tick = contracts.iloc[i]['Recipient Name']
        contractStart = contracts.iloc[i]['Start Date']
        id = contracts.iloc[i]['internal_id']
        bDay = getBuyDay(contractStart, portfolio.buySig)
        if (buy(tick, portfolio.capital*0.10, bDay, id, portfolio)):
            sell(tick, getSellDay(contractStart, tick, portfolio), id, portfolio)
        else:
            continue
    return portfolio

In [20]:
from concurrent.futures import ThreadPoolExecutor

allPortfolios = []

with ThreadPoolExecutor(max_workers=5) as executor:
    futures = [
        executor.submit(runPortfolio, contracts17.sample(150, replace=False), 2.5, 3),
    ]

    for future in futures:
        allPortfolios.append(future.result()) 
print(allPortfolios)

B394679BA253.37D2017-09-19
S394679BA264.86D2017-10-24
B1407305RTX71.38D2017-05-10


TypeError: can only concatenate str (not "NoneType") to str